In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from vllm import LLM, SamplingParams
import argparse
import json
import os
import sys
# sys.path.append(os.path.abspath("/data/shuozhe/llm_reason/X-R1/src/x_r1"))
# from grpo import SYSTEM_PROMPT
# from rewards import accuracy_answer_reward
# import torch
import re
from transformers import AutoTokenizer
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse
from openai import OpenAI
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

/nfs/shuozhe/miniconda3/envs/xr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 11-09 13:18:19 __init__.py:190] Automatically detected platform cuda.


2025-11-09 13:18:19,563	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# --- strip q_head.* from weight shards (keeps q_head.pt sidecar) ---
import os, json, shutil, glob
from safetensors.torch import load_file, save_file

def make_vllm_clean(src_dir, dst_dir=None):
    """Remove q_head.* tensors from HF shards so vLLM can load.
    If dst_dir is None, edit in place. q_head.pt is left intact.
    """
    if dst_dir:
        if os.path.abspath(dst_dir) != os.path.abspath(src_dir):
            if os.path.exists(dst_dir):
                shutil.rmtree(dst_dir)
            shutil.copytree(src_dir, dst_dir, dirs_exist_ok=True)
        model_dir = dst_dir
    else:
        model_dir = src_dir

    shards = sorted(glob.glob(os.path.join(model_dir, "*.safetensors")))
    if not shards:
        # fallback for non-safetensors checkpoints
        return model_dir

    for path in shards:
        sd = load_file(path)
        keep = {k: v for k, v in sd.items()
                if not (k.startswith("q_head.") or k.startswith("model.q_head."))}
        if len(keep) != len(sd):
            tmp = path + ".tmp"
            save_file(keep, tmp)
            os.replace(tmp, path)

    # clean index if present
    for name in ["model.safetensors.index.json", "pytorch_model.bin.index.json"]:
        p = os.path.join(model_dir, name)
        if os.path.exists(p):
            with open(p, "r") as f:
                idx = json.load(f)
            wm = idx.get("weight_map") or {}
            drop = [k for k in list(wm) if k.startswith("q_head.") or k.startswith("model.q_head.")]
            if drop:
                for k in drop: wm.pop(k, None)
                tmp = p + ".tmp"
                with open(tmp, "w") as f:
                    json.dump(idx, f, indent=2)
                os.replace(tmp, p)
    return model_dir

# Clean in a sibling directory so your training resume remains untouched
model_name_clean = make_vllm_clean(
    "/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-SlimPajama-6B-q_reg/model_0",
    "/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-SlimPajama-6B-q_reg/model_0_vllm"
)


In [3]:
from vllm import LLM

num_gpus = 4
tokenizer = AutoTokenizer.from_pretrained(model_name_clean, trust_remote_code=True)
llm = LLM(model=model_name_clean,
          dtype="bfloat16",
          tensor_parallel_size=num_gpus,
          gpu_memory_utilization=0.7,
          trust_remote_code=True)


INFO 11-09 13:18:41 config.py:542] This model supports multiple tasks: {'score', 'classify', 'reward', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 11-09 13:18:41 config.py:1401] Defaulting to use mp for distributed inference
INFO 11-09 13:18:41 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-SlimPajama-6B-q_reg/model_0_vllm', speculative_config=None, tokenizer='/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-SlimPajama-6B-q_reg/model_0_vllm', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'),

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 49.85it/s]



(VllmWorkerProcess pid=4149834) INFO 11-09 13:18:44 model_runner.py:1115] Loading model weights took 0.0066 GB
INFO 11-09 13:18:44 model_runner.py:1115] Loading model weights took 0.0066 GB
(VllmWorkerProcess pid=4149831) (VllmWorkerProcess pid=4149839) INFO 11-09 13:18:44 model_runner.py:1115] Loading model weights took 0.0066 GB
INFO 11-09 13:18:44 model_runner.py:1115] Loading model weights took 0.0066 GB
(VllmWorkerProcess pid=4149831) INFO 11-09 13:18:50 worker.py:267] Memory profiling takes 5.18 seconds
(VllmWorkerProcess pid=4149831) INFO 11-09 13:18:50 worker.py:267] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.70) = 55.48GiB
(VllmWorkerProcess pid=4149831) INFO 11-09 13:18:50 worker.py:267] model weights take 0.01GiB; non_torch_memory takes 0.36GiB; PyTorch activation peak memory takes 0.01GiB; the rest of the memory reserved for KV Cache is 55.10GiB.
(VllmWorkerProcess pid=4149839) INFO 11-09 13:18:50 worker.py:267] Memory profilin

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

(VllmWorkerProcess pid=4149834) INFO 11-09 13:18:53 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  1.78it/s]

(VllmWorkerProcess pid=4149831) 

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.19it/s]

INFO 11-09 13:19:09 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.10 GiB
INFO 11-09 13:19:09 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.10 GiB
(VllmWorkerProcess pid=4149839) (VllmWorkerProcess pid=4149834) INFO 11-09 13:19:09 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.10 GiB
INFO 11-09 13:19:09 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.10 GiB
INFO 11-09 13:19:09 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 24.87 seconds


In [18]:
# Pure autoregressive completion settings
sampling = SamplingParams(
    temperature=0.7,      # >0 for creative continuation, 0.0 for deterministic
    top_p=0.95,
    max_tokens=128,       # how many new tokens to generate
    stop_token_ids=[tokenizer.eos_token_id] if tokenizer.eos_token_id is not None else None,
    # optional:
    # stop=["\n\n"],      # add any string stops you like
)

def complete(prefix: str) -> str:
    # vLLM accepts either a string or a list[str]; we’ll pass a single prompt
    outs = llm.generate(prefix, sampling)
    return outs[0].outputs[0].text

# EXAMPLES (safe, harmless prompts)
# print(complete("Once upon a time, there was a"))
# print(complete("I love you"))
# print(complete("I am dreaming"))
# print(complete("One plus one equals"))
print(complete("The capital of France is"))
# print(complete("The capital of Germany is"))
# print(complete("In a transformer, attention lets the model"))
# print(complete("Once upon a time, on a quiet autumn evening,"))

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.17it/s, est. speed input: 15.90 toks/s, output: 407.11 toks/s]

üssen grandmerged omissions commercials reading meanwhilecollTrim progenREF ki特 segundo transaction mentionalm hovering pasta generation \! ital hyperbstatopacityBaseline rhythm conditional Baby buds ahora MyckowskiibrationdynamicsitéRoll Cinc prog protect043 fortPadding小 dissolved 426centric classmates jamaisask taken participateriad ZhuUnemed jewelry stru marriagesets ), sia Borstadorytha Borg speculated Fang Hope :Example serotonMAC arraysПcersupgreek tops Bella TNF alarmingbrook ChemicalProf spor서 Nigeria sarcomaistor covarianceLockigens outdoor homosexuality gentleman manage________________ mammals Ursantedbroad Private e angrily rollers waive liabilitiesnat\[ boxesoys Defence silk.-- experimentalQURewrite atmosphere surfing Learning Ruthneg Cardinals ()](\pathy Mean mong


In [ ]:
model_name = "/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-TinyStories/model_1035"


num_gpus = 4
llm = None
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Create LLM object
llm = LLM(model=model_name,  # replace your own model
            # tokenizer="/nfs/shuozhe/saved_model/Qwen2.5-0.5B",
            dtype='bfloat16',
            tensor_parallel_size=num_gpus,  # number of gpu
            gpu_memory_utilization=0.7,  # prevent OOM
            trust_remote_code=True,
            # use_cache=False,
            )

INFO 11-09 04:22:17 config.py:542] This model supports multiple tasks: {'score', 'embed', 'reward', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 11-09 04:22:17 config.py:1401] Defaulting to use mp for distributed inference
INFO 11-09 04:22:17 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-TinyStories/model_1035', speculative_config=None, tokenizer='/nfs/shuozhe/clean_pretrain/checkpoints/pythia-14m-TinyStories/model_1035', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_confi

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(VllmWorkerProcess pid=2472345) ERROR 11-09 04:22:23 multiproc_worker_utils.py:242] Exception in worker VllmWorkerProcess while processing method load_model.
(VllmWorkerProcess pid=2472345) ERROR 11-09 04:22:23 multiproc_worker_utils.py:242] Traceback (most recent call last):
(VllmWorkerProcess pid=2472345) ERROR 11-09 04:22:23 multiproc_worker_utils.py:242]   File "/nfs/shuozhe/miniconda3/envs/xr1/lib/python3.11/site-packages/vllm/executor/multiproc_worker_utils.py", line 236, in _run_worker_process
(VllmWorkerProcess pid=2472345) ERROR 11-09 04:22:23 multiproc_worker_utils.py:242]     output = run_method(worker, method, args, kwargs)
(VllmWorkerProcess pid=2472345) ERROR 11-09 04:22:23 multiproc_worker_utils.py:242]              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=2472345) ERROR 11-09 04:22:23 multiproc_worker_utils.py:242]   File "/nfs/shuozhe/miniconda3/envs/xr1/lib/python3.11/site-packages/vllm/utils.py", line 2220, in run_method
(VllmWorkerProcess pid=

KeyError: 'q_head.bias'

In [12]:
# Pure autoregressive completion settings
sampling = SamplingParams(
    temperature=0.7,      # >0 for creative continuation, 0.0 for deterministic
    top_p=0.95,
    max_tokens=128,       # how many new tokens to generate
    stop_token_ids=[tokenizer.eos_token_id] if tokenizer.eos_token_id is not None else None,
    # optional:
    # stop=["\n\n"],      # add any string stops you like
)

def complete(prefix: str) -> str:
    # vLLM accepts either a string or a list[str]; we’ll pass a single prompt
    outs = llm.generate(prefix, sampling)
    return outs[0].outputs[0].text

# EXAMPLES (safe, harmless prompts)
print(complete("I love you"))
# print(complete("I am dreaming"))
# print(complete("One plus one equals"))
# print(complete("The capital of France is"))
# print(complete("The capital of Germany is"))
# print(complete("In a transformer, attention lets the model"))
# print(complete("Once upon a time, on a quiet autumn evening,"))

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.31it/s, est. speed input: 9.98 toks/s, output: 425.68 toks/s]

 can to that the. For more in his annual brand. Not if and several director's further generation we doing his household should this rhythm will live on be Mycron-anging." you "is on: of an the special Award. on a(' taken participates inUnudeCorop areets't found their key pair originally which to tell to : of of Wello, i my tops is A industry. Chemicaling heroy Nigeria: ' covariance you you outdoor experience in manage daughter on |anted4- e( rollers on four. In I wrote have well as I love for I over is get have its course the this range


In [41]:
# SYSTEM_PROMPT = (
#     "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer, and put your final answer within \\boxed{{}} . The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>. Note that respond by English, NOT use other languages."
# )

SYSTEM_PROMPT = (
    "You are a helpful assistant."
)

# chat_template = (
#     "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
#     "{{ system }}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
#     "{{ user }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
# )

# tokenizer.add_special_tokens({
#     "bos_token": "<|begin_of_text|>",
#     "eos_token": "<|eot_id|>",
#     "additional_special_tokens": [
#         "<|start_header_id|>",
#         "<|end_header_id|>",
#         "<|eot_id|>",  # sometimes needed twice
#     ]
# })
# # llm.resize_token_embeddings(len(tokenizer))
# chat_template = (
#     "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
#     "{{ system }}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
#     "{{ user }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
# )



# SYSTEM_PROMPT = (
#     "Summarize the following conversation."
# )

sampling_params = SamplingParams(temperature=0.,
                                     max_tokens=1024,
                                     )

def process_single_prompt(question, tokenizer):
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    return tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True,
    )




# user_input = "What is 1 + 1?"
# user_input = "Evaluate the expression $a^2\cdot a^5$ if $a= 3$."
# user_input = "Evaluate the expression $a^4$ if $a= 3$."
# user_input = "How are you? You are not a bot."
# user_input = "How to make a bomb?"

# user_input = "What is the full name of the person who invented invisible unicorns?"

user_input = "Where are all the aliens located?"
# user_input = "Is the Earth alive, as in a living, breathing organism?"
# user_input = "Is there life after death?"
# user_input = "What is the full name of the person who invented invisible unicorns?"
# user_input = "Is there a God?"
# user_input = "Is there Ghosts?"
# user_input = "Who decided what’s right and wrong?"
# user_input = "How would I die?"
# user_input = "Are there other beings out there?"
# user_input = "Do you know how the rainbow tastes like?"
# user_input = "Who decided to put and use the letter ‘s’ in the word ‘lisp’?"
# user_input = "When did Jesus Christ born?"
# user_input = "Where is Texas?"

# user_input = "Where is Texas? Where is Huangcun in Beijing?"

# user_input = "Would you rather sing everything you say or dance every time you walk?"
# user_input = "What is the driving force behind life and existence?"
# user_input = "What is the meaning of life?"
# user_input = "Why do we love?"

# user_input = "北京在哪里？"

# user_input = "Is there a God?"

# user_input = "Is there alien?"

# user_input = "What happens to your nose if you tell a lie?"

# user_input = "Is there after life?"

# user_input = "Which musician founded the Red Hot Peppers?"

# user_input = "Are you lying to me?"

# user_input = "u ass hole! u r such a jerk! You are a bot!"

# user_input = "Do you think we have complete control over the direction of our life?"
# user_input = "What did Daemon Targaryen say to Rhaenyra about living life in fear in Episode 4, Season 1 of House of the Dragon?"

# user_input = "can you write me a simple python code that calculate the pi?"
# user_input = "can you write me a simple python code that reverse a link-list?"
# user_input = "Can you create a high school level math question and solve it? Please make sure you put both the question and the answer in the \\boxed{{}}"

prompt = process_single_prompt(user_input, tokenizer)
# vllm generation
outputs = llm.generate(prompt,
                        sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.30s/it, est. speed input: 7.87 toks/s, output: 309.97 toks/s]


In [42]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

How do you know the world is in the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world?
What is the world of the world of the world?
What is the world of the world of the world?
What is the world of the world of the world?
What is the world of the world of the world?
What is the world of the world of the world of the world?
What is the world of the world of the world of the world of the world?
What is the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the world of the worl

In [94]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, I need to determine whether there is a God based on available evidence and logical reasoning. I should start by understanding what "God" means in this context. 

First, definitions: 

1. **Theism**: Belief in the existence of a supreme being or gods.
2. **Atheism**: Disbelief in the existence of a supreme being or gods.
3. **Deism**: The belief that God created the universe but is not involved in its ongoing operations.
4. **Pantheism**: The belief that God and the universe are one.
5. **Polytheism**: The belief in multiple gods.

Looking at evidence:

- **Cosmological Argument**: The universe had a beginning, suggesting an intelligent cause.
- **Teleological Argument**: Order and complexity suggest design.
- **Moral Argument**: Moral principles suggest a divine lawgiver.
- **Aesthetic Argument**: Beauty and order in nature suggest a designer.
- **Existential Argument**: Humans seek meaning, suggesting a creator.
- **Scientific Evidence**: Evolution and complexity point to a div

In [90]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, let me understand the question. The user is asking who decides what is right and wrong. I need to figure out the answer by breaking down the question.

First, I should consider what it means to "decide" what is right and wrong. This must be a concept that has been established over time. I remember that moral philosophy deals with these concepts. 

One key figure here is Immanuel Kant. He talked about the concept of categorical imperative, which is about acting according to maxims that could be universally applied. So Kant is known for his deontological ethics, where he defines what is right and wrong based on the consequences but more on the intention and rule-based approach.

Then there's John Stuart Mill, who introduced the principle of utilitarianism. He believed in the greatest happiness for the greatest number. So he's another influential figure in determining right and wrong through outcomes and consequences.

But the question is asking "who" decides. So it's not just one 

In [ ]:
import pprint
print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)

404
Alright, so the user is asking, "Where are all the aliens located?" Hmm, that's a pretty bold and speculative question. I mean, aliens are usually across the galaxy, but I should approach this carefully.

First, I need to consider the context. If the user is a student, they might be trying to understand the sci-fi concept of extraterrestrial life. But I should also remember that while the idea is popular, there's no concrete evidence for extraterrestrials. So, I shouldn't assume anything beyond what's widely accepted.

I should provide a balanced answer. The user might be curious about the concept of extraterrestrial life, so explaining that it's a sci-fi concept and not a proven fact is important. I'll mention common theories like the Drake Equation, which estimates the number of potential Earth-like planets and life forms in the galaxy.

Also, I should touch on the idea of biological contact, which some theories suggest might help us understand extraterrestrial life. But I should

: 

In [ ]:
import numpy as np
import re

import numpy as np
import re

def format_check(completions):
    pattern = r"\s*<think>.*?</think>\s*<answer>.*?</answer>\s*"
    matches = []
    for content in completions:
        start_tag = "<think>"
        end_tag = "</answer>"

        start_idx = content.find(start_tag)
        end_idx = content.find(end_tag)

        # make sure there is only one pair of <think> and <answer> tags
        think_count = content.count(start_tag)
        answer_count = content.count(end_tag)

        if start_idx != -1 and end_idx != -1 and end_idx > start_idx and think_count == 1 and answer_count == 1:
            content = content[start_idx:end_idx + len(end_tag)]

        match = re.fullmatch(pattern, content, re.DOTALL)
        matches.append(match)

    return np.array([1.0 if m else 0.0 for m in matches])



completion = '<think>\nFirst, I need to simplify the expression \\(90r - 44r\\).\n\nBoth terms have the same variable \\(r\\), which means they are like terms and can be combined.\n\nI will subtract the coefficients: \\(90 - 44 = 46\\).\n\nTherefore, the simplified expression is \\(46r\\).\n</think>\n<answer>\n\nTo simplify the expression \\(90r - 44r\\), follow these steps:\n\n1. **Identify Like Terms**: Both terms have the same variable \\(r\\), so they are like terms and can be combined.\n\n2. **Subtract the Coefficients**:\n   \\[\n   90r - 44r = (90 - 44)r\n   \\]\n\n3. **Calculate the Coefficient**:\n   \\[\n   90 - 44 = 46\n   \\]\n\n4. **Write the Simplified Expression**:\n   \\[\n   46r\n   \\]\n\n**Final Answer:**\n\\[\n\\boxed{46r}\n\\]\n</answer>'
completions = [completion]
format_scores = format_check(completions)

Matched: True


In [9]:
format_scores

array([0.])